In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [21]:
import sys
sys.path.append('..')

import autorobot as ar
from autorobot.constants import (
    RProjType,
)

In [13]:
rb = ar.initialize()

In [14]:
rb.new(RProjType.SHELL)

In [15]:
rb.nodes.create(0, 0, 1)
rb.nodes.create(0, 1, 0)
rb.nodes.create(1, 0, 0)

In [16]:
node = rb.nodes.get('2')
print(node.to_array())

[0. 1. 0.]


In [17]:
for n in rb.nodes.select('1to3'):
    print(n.to_array())

[0. 0. 1.]
[0. 1. 0.]
[1. 0. 0.]


In [22]:
ar.distance(1, '3')

1.4142135623730951

In [19]:
with rb.nodes as nodes:
    print(nodes.get(1))

In [20]:
rb.bars.create(1, 2)